<a href="https://colab.research.google.com/github/ToonWeyens/cometml/blob/main/Image_Classification_with_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Comet

In [ ]:
# !pip install comet_ml --quiet

# Import Comet

In [1]:
import comet_ml
comet_ml.init(project_name='keras-image-classification')

COMET INFO: Comet API key is valid


# Import Dependencies

In [2]:
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Create an Experiment

In [3]:
#create an experiment with your api key
experiment = comet_ml.Experiment(api_key="sqALCFsmSwyAhS0zhRFeZdL7j")

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.ml https://www.comet.com/toonweyens/keras-image-classification/65dae46a1e2242dd9edfd24b4f4d4306



# Load Data

In [4]:
# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

60000 train samples
10000 test samples


# Define Training Parameters

In [5]:
# these will all get logged
params = {
    "batch_size": 128,
    "epochs": 2,
    "layer1_type": "Dense",
    "layer1_num_nodes": 64,
    "layer1_activation": 'relu',
    "optimizer": 'adam',
}

num_classes = 10

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)



# Define Model

In [6]:
model = Sequential()
model.add(Dense(64, activation="relu", input_shape=(784,)))
model.add(Dense(num_classes, activation="softmax"))

# print model.summary() to preserve automatically in `Output` tab
print(model.summary())

model.compile(
    loss="categorical_crossentropy", optimizer=params['optimizer'], metrics=["accuracy"]
)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                50240     
                                                                 
 dense_1 (Dense)             (None, 10)                650       
                                                                 
Total params: 50,890
Trainable params: 50,890
Non-trainable params: 0
_________________________________________________________________
None


2022-09-05 19:53:32.066478: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Log Experiment Parameters

In [7]:
experiment.log_parameters(params)
experiment.log_dataset_hash(x_train)  # creates and logs a hash of your data

# Log Training

In [8]:
# will log metrics with the prefix 'train_'
model.fit(
    x_train,
    y_train,
    batch_size=params['batch_size'],
    epochs=params['epochs'],
    verbose=1,
    validation_data=(x_test, y_test),
)

COMET INFO: Ignoring automatic log_parameter('verbose') because 'keras:verbose' is in COMET_LOGGING_PARAMETERS_IGNORE


Epoch 1/2
469/469 [==============================] - 2s 3ms/step - loss: 0.4269 - accuracy: 0.8826 - val_loss: 0.2450 - val_accuracy: 0.9309
Epoch 2/2
469/469 [==============================] - 1s 3ms/step - loss: 0.2085 - accuracy: 0.9405 - val_loss: 0.1742 - val_accuracy: 0.9483


# Log Evaluation

## Logging Evaluation Metrics

In [9]:
loss, accuracy = model.evaluate(x_test, y_test)
metrics = {"loss": loss, "accuracy": accuracy}
with experiment.test():
  experiment.log_metrics(metrics)

313/313 [==============================] - 0s 1ms/step - loss: 0.1742 - accuracy: 0.9483


## Logging the Confusion Matrix and Images 

Comet gives you the option to log Images with the `experiment.log_image` method. We're going to use this method along with our Confusion Matrix so that we can log samples from our dataset and identify misclassified images in the UI.   

In [10]:
predictions = model.predict(x_test)

313/313 [==============================] - 0s 1ms/step


In [12]:
# Logs the image corresponding to the model prediction 
def index_to_example(index):
    image_array = x_test[index].reshape(28, 28)
    image_name = "confusion-matrix-%05d.png" % index
    results = experiment.log_image(image_array, name=image_name)
    # Return sample, assetId (index is added automatically)
    return {"sample": image_name, "assetId": results["imageId"]}

LABELS = [f"class_{i}" for i in range(10)]

experiment.log_confusion_matrix(
    y_test[:100],
    predictions[:100],
    labels=LABELS,
    index_to_example_function=index_to_example,
    title="Confusion Matrix: Evaluation",
    file_name="confusion-matrix-eval.json",
)

{'web': 'https://www.comet.com/api/asset/download?assetId=a7fef9ec250f4fea90040e056c93a4ef&experimentKey=65dae46a1e2242dd9edfd24b4f4d4306',
 'api': 'https://www.comet.com/api/rest/v2/experiment/asset/get-asset?assetId=a7fef9ec250f4fea90040e056c93a4ef&experimentKey=65dae46a1e2242dd9edfd24b4f4d4306',
 'assetId': 'a7fef9ec250f4fea90040e056c93a4ef'}

# Ending an Experiment

In [13]:
experiment.end()

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/toonweyens/keras-image-classification/65dae46a1e2242dd9edfd24b4f4d4306
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [2]                 : (0.8825666904449463, 0.9405333399772644)
COMET INFO:     batch_accuracy [94]          : (0.1015625, 0.9609375)
COMET INFO:     batch_loss [94]              : (0.1226208284497261, 2.479597568511963)
COMET INFO:     epoch_duration [2]           : (1.2001186270000002, 1.8067181239999996)
COMET INFO:     loss [2]                     : (0.20852313935756683, 0.4268640875816345)
COMET INFO:     test_accuracy                : 0.9483000040054321
COMET INFO:     test_loss                    : 0.17422275245189667
COMET INFO:     val_accuracy [2]             : (0.930899977684021, 0.9483000040054321)
C